In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Fine-tuning Pipeline

In [2]:
# load libraries and model from HF
import torch
import pandas as pd
import wandb
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments
from unsloth import FastLanguageModel, is_bfloat16_supported

from _1_prompt_templates import format_data, token_dict
token_pos = 0

model_name = "codellama/CodeLlama-7b-Instruct-hf"
max_seq_length = 2048
random_seed= 3407 

timestamp = pd.Timestamp.now().strftime("%Y%m%d%H%M")
# Initialize WandB (ensure you've logged in using `wandb login`)
wandb.init(project="code-llama-finetuning", 
           name=f"fine-tune-instruct-semantic-length-generalization-ascii-desc_{timestamp}",
           config={"learning_rate": 5e-5, "num_train_epochs": 1, "max_seq_length": max_seq_length, "num_epochs": 1,})

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


wandb: Currently logged in as: priscillachyrva (priscillachyrva-university-mannheim). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
# Model configuration
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

codellama/CodeLlama-7b-Instruct-hf does not have a padding token! Will use pad_token = <unk>.


In [5]:
# if tokenizer.pad_token is None then an error will be raised I explicitly set it to '[PAD]'
if tokenizer.pad_token is None or tokenizer.pad_token == tokenizer.unk_token:
    tokenizer.pad_token = tokenizer.eos_token or "[PAD]"
    if tokenizer.pad_token == "[PAD]":
        tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    print(f"Added padding token: {tokenizer.pad_token}")

for k, v in token_dict.items():
    special_tokens = []
    if not v[0]== None:
        for token in v[token_pos]: 
            if token and token not in [
            tokenizer.eos_token,
            tokenizer.pad_token,
            tokenizer.unk_token,
            tokenizer.bos_token,
            ] and token not in special_tokens:
                special_tokens.append(token)

tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
model.resize_token_embeddings(len(tokenizer))

Added padding token: </s>


Embedding(32016, 4096, padding_idx=0)

In [ ]:
tokenized_output = tokenizer(sample_prompt, return_tensors="pt")
num_tokens = tokenized_output["input_ids"].shape[1]

print(f"Sample prompt token count: {num_tokens}")

# Check if it exceeds max_seq_length
if num_tokens > max_seq_length:
    print(f"Warning: Sample prompt exceeds {max_seq_length} tokens ({num_tokens} tokens)! Consider truncation.")
else:
    print(f"Token count within limit: {num_tokens} tokens")

In [4]:
# LoRA (16-bit) for PEFT => this means I need 16GB of VRAM when training the 7B-codelama model (Alternative: QLoRA could be specified with 8GB VRAM)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state= random_seed
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Load and Preprocess Train, Validation, and Test-Data

In [5]:
from datasets import load_dataset

In [6]:
# load the datasets and access the splits
dataset = load_dataset("ruthchy/semantic-length-generalization-logo-data-desc-ascii_35")
print(dataset)
#train_dataset, val_dataset, test_dataset = dataset["train"], dataset["validation"], dataset["test"]

#formatted_train_dataset = format_data(train_dataset, include_description=True, include_ascii=True, token_dict=token_dict, token_position=token_pos)
#formatted_val_dataset = format_data(val_dataset, include_description=True, include_ascii=True, token_dict=token_dict, token_position=token_pos)
#formatted_test_dataset = format_data(test_dataset, include_description=True, include_ascii=True, token_dict=token_dict, token_position=token_pos)

#print(formatted_train_dataset["Prompt"][0])

DatasetDict({
    train: Dataset({
        features: ['Description', 'ASCII-Art', 'Program'],
        num_rows: 8006
    })
    validation: Dataset({
        features: ['Description', 'ASCII-Art', 'Program'],
        num_rows: 997
    })
    test: Dataset({
        features: ['Description', 'ASCII-Art', 'Program'],
        num_rows: 997
    })
})


In [7]:
#from _1_prompt_temp_v2 import instruction_format, conversational_format_v1, sys_prompt
from _1_prompt_temp_v2 import instruction_format_v1, conversational_format_v1, sys_prompt

for split in dataset:
    dataset[split] = dataset[split].map(lambda x: instruction_format_v1(x, include_description=True, include_ascii=True)) 
    #dataset[split] = dataset[split].rename_column("messages", "conversations")

train_dataset, val_dataset, test_dataset = dataset["train"], dataset["validation"], dataset["test"]
train_dataset

Dataset({
    features: ['Description', 'ASCII-Art', 'Program', 'conversations'],
    num_rows: 8006
})

In [6]:
train_dataset["conversations"][0]

'Below are some instructions that describe some tasks. Write responses that appropriately complete each request.\n### Instruction: Generate a python program producing the graphic, which is described and depicted as follows:\n    The Program draws a 5 pointed star\n    Graphic:\n00000000000000000000000000000000000\n00000000000000000000000120000000000\n00000000000000000000002220000000000\n00000000000000000000031020000000000\n00000000000000000001300002000000000\n00000000000000000021000002000000000\n00000000000000000300000002000000000\n00000000000000012000000000200000100\n00000000000000310000000000300000200\n00000000000013000000000000200001100\n00000000000220000000000000020002000\n00000000003100000000000000020002000\n00000000130000000000000000020001000\n00000002100000000000000000002020000\n00000030000000000000000000002020000\n00001200000000000000000000002020000\n00031000000000000000000000000500000\n00400000000000000000000000000400000\n00031000000000000000000000000500000\n000012000000000000

In [ ]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.
### Instruction:
{INPUT}
### Python Program:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    default_system_message = sys_prompt, # optional
)

In [9]:
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM # to train model only on the generated prompts


In [10]:
response_template = "### Python Program:" # https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)



In [22]:
import yaml
def load_config(yaml_file):
    with open(yaml_file, "r") as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

cuda_devices = config["cuda"]["devices"]
# data
data_dir = config["data"]
# model
model_name = config["model"]["name"]
# training
trainer = config["training"]["trainer"]

max_seq_length = config["training"]["max_seq_length"]
learning_rate = float(config["training"]["learning_rate"])
train_epochs = config["training"]["train_epochs"]
per_device_batch_size = config["training"]["per_device_batch_size"]
gradient_accumulation_steps = config["training"]["gradient_accumulation_steps"]
save_steps = config["training"]["save_steps"]
eval_steps = config["training"]["eval_steps"]
logging_steps = config["training"]["logging_steps"]
random_seed = config["training"]["random_seed"]

trainer

'trainer_1'

In [23]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["Description"])):
        formated_input = f"Generate a python program producing the graphic, which is described and depicted as follows:\n    The Program draws {example['Description'][i]}\n    Graphic:\n{example['ASCII-Art'][i]}\n"
        text = f"{formated_input}\n ### Answer: {example['Program'][i]}"
        output_texts.append(text)
    return output_texts


In [24]:
trainer = SFTTrainer(
    model,
    #contains tokenizer)
    data_collator=collator,          # tokenizer or collator (bc collator contains tokenizer)
    formatting_func=formatting_prompts_func, # (Callable[[dict], str]) — The function used to format the input data. It should return a string.
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args = SFTConfig(
        output_dir = f"./results/fine-tune-{model_name.split('/')[-1]}-_{timestamp}",
        # pre-processing
        max_seq_length = max_seq_length,
        packing = False,
        dataset_num_proc =4,
        # training parameters
        learning_rate = learning_rate,
        num_train_epochs = train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        per_device_train_batch_size = per_device_batch_size,
        per_device_eval_batch_size = per_device_batch_size,
        # reporting and logging
        report_to = ["wandb"],
        push_to_hub = True,
        hub_model_id = f"fine-tune-{model_name.split('/')[-1]}-_{timestamp}",
        logging_strategy = "steps",
        logging_steps = logging_steps,
        # checkpointing
        save_strategy = "steps",
        save_steps = save_steps,
        # evaluation
        eval_strategy = "steps",
        eval_steps = eval_steps,
        # optimization
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),    
        # ensure reproducibility
        seed = random_seed,
        data_seed = random_seed,
        full_determinism=True
        )
    )

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,006 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 250
 "-____-"     Number of trainable parameters = 39,976,960


ZeroDivisionError: division by zero

In [15]:
sys_prompt = """Your task is to draw simple black and white graphics with the custom library. DO NOT USE THE BUILT-IN TURTLE LIBRARY.
You will use a custom turtle library, similar to the built-in library, which is sufficient for all tasks.

Here are all the available functions in the custom turtle library:
- forward(x): move forward x pixels
- left(theta): rotate left by theta degrees
- right(theta): rotate right by theta degrees
- penup(): stop drawing
- pendown(): start drawing
- teleport(x, y, theta): move to position (x, y) with angle theta
- heading(): get the current angle of the turtle
- isdown(): check if the pen is down
- embed(program, local vars): runs the code in program using the current context and teleports back to the original position. Allows you to nest programs. Implementationally, embed gets the turtle state (is down, x, y, heading), executes program, then returns to the original state."""


def conversational_format(example):
    return {
        "messages": [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": f"Generate a python program producing the graphic, which is described and depicted as follows:\n The Program draws {example['Description']}\n Graphic:\n{example['ASCII-Art']}\n"},
            {"role": "assistant", "content": example["Program"]}
        ]
    }

In [16]:
for split in dataset:
    dataset[split] = dataset[split].map(lambda x: format_example(x))

# Print out a sample to confirm the changes
print(dataset["train"][0])

Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

{'Description': 'a 5 pointed star', 'ASCII-Art': '00000000000000000000000000000000000\n00000000000000000000000120000000000\n00000000000000000000002220000000000\n00000000000000000000031020000000000\n00000000000000000001300002000000000\n00000000000000000021000002000000000\n00000000000000000300000002000000000\n00000000000000012000000000200000100\n00000000000000310000000000300000200\n00000000000013000000000000200001100\n00000000000220000000000000020002000\n00000000003100000000000000020002000\n00000000130000000000000000020001000\n00000002100000000000000000002020000\n00000030000000000000000000002020000\n00001200000000000000000000002020000\n00031000000000000000000000000500000\n00400000000000000000000000000400000\n00031000000000000000000000000500000\n00001200000000000000000000002020000\n00000030000000000000000000002020000\n00000002100000000000000000002020000\n00000000130000000000000000020001000\n00000000003100000000000000020002000\n00000000000220000000000000020002000\n0000000000001300000000000

In [17]:
sys_prompt = """Your task is to draw simple black and white graphics with the custom library. DO NOT USE THE BUILT-IN TURTLE LIBRARY.
You will use a custom turtle library, similar to the built-in library, which is sufficient for all tasks.

Here are all the available functions in the custom turtle library:
- forward(x): move forward x pixels
- left(theta): rotate left by theta degrees
- right(theta): rotate right by theta degrees
- penup(): stop drawing
- pendown(): start drawing
- teleport(x, y, theta): move to position (x, y) with angle theta
- heading(): get the current angle of the turtle
- isdown(): check if the pen is down
- embed(program, local vars): runs the code in program using the current context and teleports back to the original position. Allows you to nest programs. Implementationally, embed gets the turtle state (is down, x, y, heading), executes program, then returns to the original state."""

# Instruction Format (alpaca style)
alpaca_prompt = """Below is an instruction that describes a task, paired with  an input that provides further context. Write a response that appropriatley completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [18]:
def format_prompt(example, include_description=False, include_ascii=False):
    # User Instruction
    if not (include_description or include_ascii):
        raise ValueError("At least one of include_description or include_ascii must be True.")
    
    user_instruction = "Generate a python program producing the graphic, which is"
    if include_description and include_ascii:
        user_instruction += " described and depicted"
    elif include_description:
        user_instruction += " described"
    elif include_ascii:
        user_instruction += " depicted"
    
    user_instruction += " as follows:\n"
    if include_description:
        user_instruction += f"    The Program draws {example['Description']}\n"
    if include_ascii:
        user_instruction += f"    Graphic:\n{example['ASCII-Art']}\n"
    
    formatted_text = alpaca_prompt.format(user_instruction, example["Description"], example["ASCII-Art"])
    return formatted_text

In [19]:
for split in dataset:
    dataset[split] = dataset[split].map(lambda x: format_prompt(x))

# Print out a sample to confirm the changes
print(dataset["train"][0])

Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

ValueError: At least one of include_description or include_ascii must be True.

In [11]:
def inquery_tokenlength(df):
    # Calculate token lengths for 'Input' and 'Program' columns
    input_lengths = [len(tokenizer.tokenize(example['Prompt'])) for example in df]
    program_lengths = [len(tokenizer.tokenize(example['Program'])) for example in df]

    # Calculate average and maximum lengths
    avg_input_length = sum(input_lengths) / len(input_lengths)
    min_input_length = min(input_lengths)
    max_input_length = max(input_lengths)

    avg_program_length = sum(program_lengths) / len(program_lengths)
    min_program_length = min(program_lengths)
    max_program_length = max(program_lengths)

    return {
        "min_input_length": min_input_length,
        "avg_input_length": avg_input_length,
        "max_input_length": max_input_length,
        "min_program_length": min_program_length,
        "avg_program_length": avg_program_length,
        "max_program_length": max_program_length,
    }

# Print token lengths for ASCII-art + Description dataset
print("ASCII-art + Description Dataset:")
lengths = inquery_tokenlength(test_dataset)
for key, value in lengths.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

ASCII-art + Description Dataset:
Min Input Length: 1585
Avg Input Length: 1600.9087261785355
Max Input Length: 1607
Min Program Length: 106
Avg Program Length: 148.1654964894684
Max Program Length: 244


In [7]:
def tokenize_function(examples):
    # Tokenize the prompt (input) -> Returns lists of lists (batch mode)
    prompt_encodings = tokenizer(
        examples["Prompt"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
    )

    # Tokenize the program (ground truth output) -> Returns lists of lists (batch mode)
    program_encodings = tokenizer(
        examples["Program"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
    )

    # Concatenate input_ids and attention_mask for each example in the batch
    input_ids = [
        prompt + program for prompt, program in zip(prompt_encodings["input_ids"], program_encodings["input_ids"])
    ]
    attention_mask = [
        prompt + program for prompt, program in zip(prompt_encodings["attention_mask"], program_encodings["attention_mask"])
    ]

    # Create labels: Mask the prompt part with -100
    labels = [
        [-100] * len(prompt) + program for prompt, program in zip(prompt_encodings["input_ids"], program_encodings["input_ids"])
    ]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels  # Model learns only from the Program
    }

# Apply the tokenizer to the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["Prompt", "Program"], num_proc=4)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["Prompt", "Program"], num_proc=4)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["Prompt", "Program"], num_proc=4)

Map (num_proc=4):   0%|          | 0/8006 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/997 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/997 [00:00<?, ? examples/s]

In [ ]:
FastLanguageModel.for_inference(model)

In [26]:
prompt = """<s>[INST]<<SYS>>
Your task is to draw simple black and white graphics with the custom library. DO NOT USE THE BUILT-IN TURTLE LIBRARY.
You will use a custom turtle library, similar to the built-in library, which is sufficient for all tasks.

Here are all the available functions in the custom turtle library:
- forward(x): move forward x pixels
- left(theta): rotate left by theta degrees
- right(theta): rotate right by theta degrees
- penup(): stop drawing
- pendown(): start drawing
- teleport(x, y, theta): move to position (x, y) with angle theta
- heading(): get the current angle of the turtle
- isdown(): check if the pen is down
- embed(program, local vars): runs the code in program using the current context and teleports back to the original position. Allows you to nest programs. Implementationally, embed gets the turtle state (is down, x, y, heading), executes program, then returns to the original state.
<</SYS>>

You will be given a query and have to produce a program. Begin your program with a comment that explains your reasoning. For example, you might write:
# Thought: the query asks for a line, so I will use the forward() function.
Example:

Query: Draw a greek spiral with 6 turns 

Python Program:

[/INST]
"""
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # If using GPU
outputs = model.generate(**inputs, max_length=4000)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Your task is to draw simple black and white graphics with the custom library. DO NOT USE THE BUILT-IN TURTLE LIBRARY.
You will use a custom turtle library, similar to the built-in library, which is sufficient for all tasks.

Here are all the available functions in the custom turtle library:
- forward(x): move forward x pixels
- left(theta): rotate left by theta degrees
- right(theta): rotate right by theta degrees
- penup(): stop drawing
- pendown(): start drawing
- teleport(x, y, theta): move to position (x, y) with angle theta
- heading(): get the current angle of the turtle
- isdown(): check if the pen is down
- embed(program, local vars): runs the code in program using the current context and teleports back to the original position. Allows you to nest programs. Implementationally, embed gets the turtle state (is down, x, y, heading), executes program, then returns to the original state.
 

You will be given a query and have to produce a program. Begin your program with a comment t

## Define the Fine-Tuning Trainer

In [8]:
# Define a base TrainingArguments
training_args = TrainingArguments(
    output_dir="./results/02_expt",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    learning_rate=5e-5,
    #max_new_tokens=300, # the longest Program in test is 244 so 300 is enough
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    report_to=["wandb"],
    hub_model_id="ruthchy/02_expt_code-llama-ascii-desc", 
    push_to_hub=True,
    logging_steps = 1,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    max_seq_length=max_seq_length,
)

### Run Fine-tuning

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,006 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 4
\        /    Total batch size = 128 | Total steps = 62
 "-____-"     Number of trainable parameters = 39,976,960
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


In [ ]:
results_1 = trainer_1.evaluate(tokenized_test_dataset_1)
wandb.log({"Experiment 1 Results": results_1})

In [ ]:
def convert_logits_to_str(predictions):
    predicted_token_ids = torch.tensor(predictions.predictions).argmax(dim=-1)
    predicted_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_token_ids]


In [2]:
import sys
import os
original_sys_path = sys.path.copy()
try:
    synthetic_data_path = os.path.abspath("synthetic_data")
    sys.path.append(synthetic_data_path)
    from _4_logo_graphic_generator_v1 import PseudoProgramInterpreter as PseudoProgramInterpreter_v1
    from _5_ascii_processor import ASCIIProcessor
finally:
    sys.path = original_sys_path

In [ ]:
model_path = "ruthchy/01_expt_code-llama-ascii"


In [ ]:
tokenized_test_dataset_copy_1 = tokenized_test_dataset_1.copy()

# Remove the "Program" column since it represents the ground truth to be predicted
tokenized_test_dataset_1 = tokenized_test_dataset_1.remove_columns("Program")

# Predict on the test dataset
predictions = trainer_1.predict(tokenized_test_dataset_1)

# Convert logits to readable text
predicted_token_ids = torch.tensor(predictions.predictions).argmax(dim=-1)
predicted_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_token_ids]

# Add predictions to the dataset
tokenized_test_dataset_1 = tokenized_test_dataset_1.add_column("predicted_output", predicted_texts)


In [ ]:
# Train and evaluate
# Fine-tune Input-Experiment 1: ASCII-art
trainer_1.train()
results_1 = trainer_1.evaluate(tokenized_test_dataset_1)
wandb.log({"Experiment 1 Results": results_1})

# Fine-tune Input-Experiment 2: ASCII-art + Description
trainer_2.train()
results_2 = trainer_2.evaluate(tokenized_test_dataset_2)
wandb.log({"Experiment 2 Results": results_2})

print("Experiment 1 Results:", results_1)
print("Experiment 2 Results:", results_2)

## Evaluate the Fine-Tuned Models

In [ ]:
# Evaluate Experiment 1
results_1 = trainer_1.evaluate(tokenized_test_dataset_1)
print("Experiment 1 Results:", results_1)

# Evaluate Experiment 2
results_2 = trainer_2.evaluate(tokenized_test_dataset_2)
print("Experiment 2 Results:", results_2)